In [9]:
#Right now we could maybe widdle down to movies above a certain rating + number of ratings

In [2]:
import pandas as pd
import csv
import time
import numpy as np
import itertools

In [104]:
df_ratings = pd.read_csv('data/ratings.tsv',sep = '\t')
titles = pd.read_csv('data/titles.tsv',sep='\t')
names = pd.read_csv('data/names.tsv',sep = '\t')
movies = titles[titles['titleType']=='movie']
movies = list(movies['tconst'].unique())

C:\Users\Matt\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [105]:
names.head()


,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0050419,tt0053137,tt0072308,tt0031983"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0037382,tt0071877,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0049189,tt0057345,tt0054452,tt0056404"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0077975,tt0072562,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050976,tt0050986,tt0060827,tt0069467"


In [91]:
##get a list of movies based on a rating threshold and year threshold
def filterMovies(ratingThreshold,yearThreshold,dfratings,dftitles):
    avgVotes = np.mean(df_ratings['numVotes'])
    qualifiedMovies = []
    len(df_ratings)
    boolean_series = dfratings.tconst.isin(movies)
    dfratings = dfratings[boolean_series]
    for row in dfratings.iterrows():
        if row[1]['averageRating'] >= ratingThreshold and row[1]['numVotes'] >= avgVotes:
            qualifiedMovies.append(row[1]['tconst'])
    for movie in qualifiedMovies:
        year = (list(dftitles[dftitles['tconst']==movie]['startYear'])[0])
        if type(year) == str:
            year = int(year)
        if year >= yearThreshold:
            qualifiedMoviesFinal.append(movie)
    return qualifiedMovies

100


In [112]:
qualifiedMovies = filterMovies(7.5,2000,df_ratings,titles)
print(len(qualifiedMovies))

5008


In [113]:
#find the frequent actors in the list of movies that meets the support threshold
def findFreqActors(qMovies,supportThreshold):
    count = 0
    freqActors = dict()
    for chunk in pd.read_csv('data/principals.tsv' ,sep='\t', chunksize = 1000000, usecols = ['tconst','category','nconst']):
        chunk = chunk[chunk['category']=='actor']
        chunk = chunk.groupby('tconst')
        for name, group in chunk:
            if name not in qMovies:
                continue
            else:
                actors = group['nconst'].unique()
                for actor in actors:
                    if actor in freqActors:
                        freqActors[actor][0] += 1
                        freqActors[actor][1].append(name)
                    else:
                        freqActors[actor] = [1,[name]]
    for actor in freqActors:
        if freqActors[actor][0] >= supportThreshold:
            freqActorsFinal[actor] = freqActors[actor]
    return freqActors

In [114]:
freqActors = findFreqActors(qualifiedMovies,2)

In [116]:
print(len(freqActors))

7018


In [118]:
def findPairs():
    freq_pairs = dict()
    for chunk in pd.read_csv('data/principals.tsv' ,sep='\t', chunksize = 1000000, usecols = ['tconst','category','nconst']):
        chunk = chunk[chunk['category']=='actor']
        chunk = chunk.groupby('tconst')
        for name, group in chunk:
            freqActorsGroup = [actor for actor in group['nconst'] if actor in freqActorsFinal]
            freqPairsGroup = list(itertools.combinations(freqActorsGroup,2))
            for pair in freqPairsGroup:
                if pair in freq_pairs:
                    freq_pairs[pair][0] += 1
                    freq_pairs[pair][1].append(name)
                else:
                    freq_pairs[pair] = [1,[name]]
        freq_pairs = dict(sorted(freq_pairs.items(), key=lambda item: item[1],reverse=True))
        freq_pairsFinal = dict()
        for pair in freq_pairs:
            if (pair[1],pair[0]) not in freq_pairsFinal:
                freq_pairsFinal[pair] = freq_pairs[pair]
        return freq_pairsFinal

In [119]:
freq_pairs = findPairs()

In [126]:
print(len(freq_pairs))
freqpairsfinal = dict()
for pair in freq_pairs:
    if (freq_pairs[pair][0] >= 4):
        freqpairsfinal[pair] = freq_pairs[pair]
    #print(getName(pair[0]),getName(pair[1]))
for pair in freqpairsfinal:
    print(getName(pair[0]),getName(pair[1]))

8049
Stan Laurel Oliver Hardy
Bud Abbott Lou Costello
Toshirô Mifune Takashi Shimura
Terence Hill Bud Spencer
Stephen Chow Man-Tat Ng
Toshio Furukawa Akira Kamiya
Basil Rathbone Nigel Bruce
Charles Chaplin Henry Bergman
Groucho Marx Harpo Marx
Groucho Marx Chico Marx
Totò Peppino De Filippo
John Wayne Ward Bond
Anil Kapoor Anupam Kher
William Shatner Leonard Nimoy
Andy Lau Man-Tat Ng
Paolo Villaggio Gigi Reder
Lew Ayres Lionel Barrymore
Buster Keaton Joe Keaton
Amitabh Bachchan Amjad Khan
Amitabh Bachchan Shatrughan Sinha
Kôji Tsuruta Tetsurô Tanba
Ashok Kumar Sunil Dutt
Sunil Dutt Pran
Warren William Eric Blore
Boris Karloff Bela Lugosi
Harpo Marx Chico Marx
Amitabh Bachchan Pran
Sunil Dutt Shatrughan Sinha
Marcello Mastroianni Michel Piccoli
Alfredo Landa José Sacristán
Kôji Tsuruta Tomisaburô Wakayama
Jack Lemmon Walter Matthau
Alberto Sordi Vittorio De Sica
Peter Sellers Herbert Lom
Gary Cooper Walter Brennan
András Kern György Cserhalmi
Diego Abatantuono Claudio Bisio
Jackie Shrof

In [121]:
def getName(nconst):
    return list(names[names['nconst']==nconst]['primaryName'])[0] 
def getSoloMovies(name1,name2,pairsDict,soloDict):
    name1solomovies = []
    name2solomovies = []
    for movie in soloDict[name1][1]:
        if movie not in pairsDict[(name1,name2)][1]:
            name1solomovies.append(movie)
    for movie in soloDict[name2][1]:
        if movie not in pairsDict[(name1,name2)][1]:
            name2solomovies.append(movie)
    return name1solomovies,name2solomovies
def getRatings(movies):
    ratings = []
    if len(movies) == 0:
        return 0
    for movie in movies:
        ratings.append(list(df_ratings[df_ratings['tconst']== movie]['averageRating'])[0])
    rating = np.mean(ratings)
    return rating

C:\Users\Matt\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
